# Quiver plots to display current vectors

To plot vector quantities such as currents from the hydrodynamic model results, we can use a [quiver plot in matplotlib](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html).  The `u` and `v` variables from the hydrodynamic model results contain the *x* and *y* components of the current vector for each cell in the dataset.

In [ ]:
# Preparation: import required libraries
import emsarray
from emsarray import plot

import matplotlib.pyplot
import matplotlib.quiver
from shapely import box, points

# Preparation: access temperature and current vector data at the surface for a single timestamp of eReefs hydrodynamic model results dataset via OPENDAP
dataset_url = "https://thredds.nci.org.au/thredds/dodsC/fx3/GBR4_H4p0_ABARRAr2_OBRAN2020_FG2Gv3_Dhnd.ncml"
dataset = emsarray.open_dataset(dataset_url, decode_timedelta=False)
surface_currents = dataset.ems.select_variables(["temp", "u", "v"]).isel(time=-1, k=-1)

# Define a function to prepare a figure which includes the temperature variable as a scalar base layer
def make_vectors_base_figure():   
    figure = matplotlib.pyplot.figure(figsize=(8, 8), layout="constrained")
    axes = figure.add_subplot(projection=dataset.ems.data_crs)

    # Add coastline and gridline markers for context
    plot.add_coast(axes)
    plot.add_gridlines(axes)
    
    # Add the temperature variable as a scalar base layer
    temp = surface_currents["temp"]
    temp_layer = dataset.ems.make_poly_collection(temp, cmap="coolwarm", clim=(27, 30), edgecolor="face")
    temp_artist = axes.add_collection(temp_layer)
    figure.colorbar(temp_artist, label="°C")
    
    return axes


Plotting every cell is straight forward and works nicely for small areas:

In [ ]:
# Prepare a new set of axes for this example
all_vectors_axes = make_vectors_base_figure()

# Add a quiver plot of surface currents at every cell as an overlay
surface_currents_quiver = dataset.ems.make_quiver(
    all_vectors_axes, 
    surface_currents["u"], surface_currents["v"],
    scale=20, scale_units="width")
all_vectors_axes.add_collection(surface_currents_quiver)

# Zoom to a small area over the Whitsundays
view_box = box(148.7, -20.4, 149.6, -19.8)
all_vectors_axes.set_extent(plot.bounds_to_extent(view_box.bounds))
all_vectors_axes.set_aspect("equal", adjustable="datalim")
all_vectors_axes.set_title("Surface water temperature and currents near the Whitsundays")


&nbsp;

Unfortunately if you zoom out to the full extent of an eReefs dataset, the arrows for the current vectors are larger than the rendered size of their grid cells and so all overlap one another. This effect makes the resulting plot a confusing mess!

In [ ]:
# Change the existing plot to zoom out to show the entire model domain
all_vectors_axes.autoscale()
all_vectors_axes.set_aspect("equal", adjustable="datalim")
all_vectors_axes.set_title("A bad plot of surface water temperature and currents over the entire model domain")
all_vectors_axes.figure

&nbsp;

One possible solution to this problem for gridded datasets like eReefs is to sample the current data at regular intervals to display only a *subset* of the vectors:

In [ ]:
import numpy
import xarray

# Make an empty array of the same shape as the data, then select every nth cell in there
samples = xarray.DataArray(numpy.full(dataset.ems.grid_size[dataset.ems.default_grid_kind], False))
samples = dataset.ems.wind(samples)
samples[::10, ::10] = True
samples = dataset.ems.ravel(samples)

# Select the (x, y) coordinates and the (u, v) components of the sampled cells
sampled_currents = dataset.ems.select_variables(["temp", "u", "v"]).isel(time=-1, k=-1)
sampled_x, sampled_y = dataset.ems.face_centres[samples].T
sampled_u = dataset.ems.ravel(sampled_currents["u"]).values[samples]
sampled_v = dataset.ems.ravel(sampled_currents["v"]).values[samples]

# Prepare a new set of axes for this example
sampled_vectors_axes = make_vectors_base_figure()

# Add a quiver plot of currents at our sampled cells as an overlay
# You can tweak the `scale` value as needed to make the arrows appear
# a useful size: smaller numbers result in larger arrows
sampled_currents_quiver = matplotlib.pyplot.quiver(
    sampled_x, sampled_y, 
    sampled_u, sampled_v,
    scale=30, scale_units="width")
sampled_vectors_axes.add_collection(sampled_currents_quiver)

# These sampled current vectors should render just fine at full extent
sampled_vectors_axes.autoscale()
sampled_vectors_axes.set_aspect("equal", adjustable="datalim")
sampled_vectors_axes.set_title("Surface water temperature and sampled currents over the entire model domain")

&nbsp;

An alternate solution is to plot vectors at regular geospatial points across the domain. This means that the vector locations are not tied to the grid geometry, which can result in arrows that are more evenly positioned on the plot.

In [ ]:
import numpy
import pandas

# Generate a mesh of points across the model domain
domain = box(*dataset.ems.bounds)
x = numpy.arange(domain.bounds[0], domain.bounds[2], 0.4)
y = numpy.arange(domain.bounds[1], domain.bounds[3], 0.4)
xx, yy = numpy.meshgrid(x, y)
points = pandas.DataFrame({
    'x': xx.flatten(),
    'y': yy.flatten(),
})

# Extract the surface current components at these locations
surface_currents.load()
point_currents = emsarray.operations.point_extraction.extract_dataframe(
    surface_currents, points, ('x', 'y'), missing_points='drop')

# Prepare a new set of axes for this example
point_vectors_axes = make_vectors_base_figure()

# Add a quiver plot of currents at our calculated points as an overlay
# You can tweak the `scale` value as needed to make the arrows appear
# a useful size: smaller numbers result in larger arrows
point_currents_quiver = matplotlib.pyplot.quiver(
    point_currents['x'], point_currents['y'], 
    point_currents['u'], point_currents['v'],
    scale=30, scale_units="width")
point_vectors_axes.add_collection(point_currents_quiver)

# These currents-at-a-point should also render just fine at full extent
point_vectors_axes.autoscale()
point_vectors_axes.set_aspect("equal", adjustable="datalim")
point_vectors_axes.set_title("Surface water temperature and currents across the entire model domain")